In [60]:
# Main Function    
def get_geolocation_data(address_string,format="XML",country="ALL",types=True):
    address='_'.join(address_string.split(' '))
    import requests
    if format=="JSON":
        data_return=data_json(address,country,types)
        return data_return
    elif format=="XML":
        data_return=data_XML(address,country,types)
        return data_return
    else: return None
    
# Extracting the information from JSON
def data_json(address,country,types):
    import requests
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s" % (address)
    response=requests.get(url)
    if response.status_code==200:
        data_json=response.json()
        if country=="ALL":
            data_return=[]
            for i in range(len(data_json['results'])): 
                if types==True:
                    data_return.append((data_json['results'][i]['formatted_address'],data_json['results'][i]['geometry']['location']['lat'],data_json['results'][i]['geometry']['location']['lng'],data_json['results'][i]['types']))
                else:data_return.append((data_json['results'][i]['formatted_address'],data_json['results'][i]['geometry']['location']['lat'],data_json['results'][i]['geometry']['location']['lng']))
        else:
            data_return=[]
            for i in range(len(data_json['results'])): 
                if data_json['results'][i]['address_components'][6]['long_name']==country:
                    if types==True:data_return.append((data_json['results'][i]['formatted_address'],data_json['results'][i]['geometry']['location']['lat'],data_json['results'][i]['geometry']['location']['lng'],data_json['results'][i]['types']))
                    else:data_return.append((data_json['results'][i]['formatted_address'],data_json['results'][i]['geometry']['location']['lat'],data_json['results'][i]['geometry']['location']['lng']))
    return data_return

# Extracting the information from XML
def data_XML(address,country,types):
    import requests
    url="https://maps.googleapis.com/maps/api/geocode/xml?address=%s" % (address)
    response=requests.get(url)
    data=response.content
    from lxml import etree
    root = etree.XML(data)
    etree.tostring(root, pretty_print=True).decode("utf-8")
    results=root.findall('result')
    data_return=[]
    for thing in results:
        formatted_address=thing.find('formatted_address').text
        lat=thing.find('geometry/location/lat').text
        lng=thing.find('geometry/location/lng').text
        type_0=[]
        for type_1 in thing.findall('type'):
            type_0.append(type_1.text)
        for type_country in thing.findall('address_component'):
            if type_country.find('type').text=='country':
                country_single=type_country.find('long_name').text
        result_single=(formatted_address,lat,lng,type_0,country_single)
        data_return.append(result_single)
    if country=="ALL":
        if types==True:
            return data_return
        else:
            data_notype=[]
            for thing in data_return:
                data_notype.append(thing[:-2])
            return data_notype
    else:
        data_onecountry=[]
        for thing in data_return:
            if thing[-1]==country:
                if types==True:
                    data_onecountry.append(thing)
                else:data_onecountry.append(thing[:-2])
                return data_onecountry    

In [61]:
get_geolocation_data("Columbia University, New York, NY",format="JSON",country="United States",types=True)

[('116 St - Columbia University, New York, NY 10027, USA',
  40.807722,
  -73.96410999999999,
  ['establishment', 'point_of_interest', 'subway_station', 'transit_station']),
 ('Schapiro Center, New York, NY 10027, USA',
  40.8097368,
  -73.9608715,
  ['premise'])]

In [62]:
get_geolocation_data("Columbia University, New York, NY",format="JSON",country="ALL",types=True)

[('116 St - Columbia University, New York, NY 10027, USA',
  40.807722,
  -73.96410999999999,
  ['establishment', 'point_of_interest', 'subway_station', 'transit_station']),
 ('Schapiro Center, New York, NY 10027, USA',
  40.8097368,
  -73.9608715,
  ['premise'])]

In [63]:
get_geolocation_data("Columbia University, New York, NY",format="JSON",country="ALL",types=False)

[('116 St - Columbia University, New York, NY 10027, USA',
  40.807722,
  -73.96410999999999),
 ('Schapiro Center, New York, NY 10027, USA', 40.8097368, -73.9608715)]